In [ ]:
# Adding all the code

#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from numpy import pi


def initilizeQubits(qc, value, reg, n):
    for i in range(n):
        if (value[i] == "1"):
            qc.x(reg[n-(i+1)])
            
"""
Load operation - Creates a superposition of all rows of a matrix

Takes the state |i>|0> --> |i>|fi> where fi is a row vectors of a matrix F

Puts the result into a superposition of all possible vector rows from F, where observing
|i> and |fi> will give you the row vectors fi .

Qubits [0 : iLength] hold the register for |i>
Qubits [iLength : 2*iLength] are ancilla bits for holding the NOT of |i>
Qubits [2*iLength : 2*iLength+(k*n)] hold the result vector |fi> and should initially be 0
Qubits [2*iLength + (k*n) : 2*iLength + n*k + n*k*2^iLength] hold the database for the matrix f
    
Parameters: iLength : Length of the register |i>
            n : Length of one element of the database
            k : Number of elements in a row of the database
            
Total qubits = 2*iLength + n*k + n*k*2^iLength
"""

def load(n, iLength, k):
    totalQubits = 2*iLength + n*k + n*k*2**iLength
    qc = QuantumCircuit(totalQubits)
    # ccx for every bit that requires a 1
    for i in range(0, iLength):
        qc.cx(i, i+iLength)
        qc.x(i+iLength)
        
    for i in range(0, 2**iLength):
        gates = []
        copy = i
        for j in range(iLength-1, -1, -1):
            if (copy - 2**j >= 0):
                copy -= 2**j
                gates.append(j)
            else:
                gates.append(j+iLength)
        for l in range(0, n*k):
            fPosition = 2*iLength + n*k + i*n*k + l
            gates.append(fPosition)
            
            qc.mct(gates, 2*iLength + l)
            
            gates.pop()      
        
    
    # Reverting the ancilla bits back to 0
    for i in range(0, iLength):
        qc.x(i+iLength)
        qc.cx(i, i+iLength)
        
    U_Load = qc.to_gate()
    U_Load.name = "U_Load"
#     print("Depth: ", qc.depth())
    return U_Load

def signedCummulativeAdder(n):
    totalQubits = 3*n
    qc = QuantumCircuit(totalQubits)
    
    # Adding A and B and Carry
    for i in range(n-1):
        qc.ccx(i, n+i, 2*n+1+i)
        qc.cx(i, n+i)
        qc.ccx(2*n+i, n+i, 2*n+1+i)
        
    qc.cx(n-1, 2*n-1)
    
    # Reversing regC to 0
    #Reversing the gate operation performed on b[n-1]
    qc.cx(3*n-1, 2*n-1)
    #Reversing the gate operations performed during the carry gate implementations
    #This is done to ensure the sum gates are fed with the correct input bit states
    for i in range(n-1):
        qc.ccx(3*n-2-i, 2*n-2-i, 3*n-1-i)
        qc.cx(n-2-i, 2*n-2-i)
        qc.ccx(n-2-i, 2*n-2-i, 3*n-1-i)
        #These two operations act as a sum gate; if a control bit is at                
        #the 1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(3*n-2-i, 2*n-2-i)
        qc.cx(n-2-i, 2*n-2-i)
        
    U_Adder = qc.to_gate()
    U_Adder.name = "U_SignedAdder"
#     print("Adder Depth:", qc.depth())
    return U_Adder

"""
2's Complement Circuit

Given a bit string s, return -s in 2's complement form, by inverting the bits and adding 1

Qubits [0 : n] are for the bit string s
Qubits [n : 2*n] are for the 2's complement result -s
Qubits [2*n : 3*n] are for carry bits for the addition
Qubits [3*n : 4*n] are for storing the 1 for the addition

Total Qubits required = 4*n

Paramters: n

"""
def twosComplement(n):
    totalQubits = 4*n
    qc = QuantumCircuit(totalQubits)
    # Copy s to reg holding -s
    for i in range(0, n):
        qc.cx(i, n+i)
        # Invert -s
        qc.x(n+i)
    # Store the 1
    qc.x(3*n)
    
    qc.append(signedCummulativeAdder(n), 
              list(range(3*n, 4*n))
              + list(range(n, 2*n))
              + list(range(2*n, 3*n))
             )
    # Reverse the 1
    qc.x(3*n)
    U_TwosComplement = qc.to_gate()
    U_TwosComplement.name = "U_TwosComplement"
    return U_TwosComplement

def controlledTwosComplement(n):
    totalQubits = 3*n+1
    qc = QuantumCircuit(totalQubits)
    # Copy s to reg holding -s
    for i in range(0, n):
        qc.cx(3*n, i)
    # Store the 1
    qc.cx(3*n, 2*n)
    
    qc.append(signedCummulativeAdder(n), 
              list(range(2*n, 3*n))
              + list(range(0, n))
              + list(range(n, 2*n))
             )
    # Reverse the 1
    qc.cx(3*n, 2*n)
    U_TwosComplement = qc.to_gate()
    U_TwosComplement.name = "U_ControlledTwosComplement"
    return U_TwosComplement

"""
Signed Shift Adder Multiplication Circuit

Multiplies two n bit registers regA and regB using a shift adder and
stores the result in a 2*n bit output register.

Qubits [0 : n] are for regA
Qubits [n : 2*n] are for regB
Qubits [2*n : 4*n] are for an ancilla register
Qubits [4*n : 6*n] are for the output register
Qubits [6*n : 8*n] are for the carry register regC
Qubits [8*n : 8*n+2] are for more ancilla bits

Total Qubits required = 8*n + 2

Paramters: n

"""

def signedMultiplier(n, inverse=False):
    totalQubits = 8*n+2
    qc = QuantumCircuit(totalQubits)
    
    # Checking if either are negative numbers
    
    qc.cx(n-1, 8*n)
    qc.cx(2*n-1, 8*n+1)
    
    qc.append(controlledTwosComplement(n), 
              list(range(0, n))
              + list(range(6*n, 7*n))
              + list(range(2*n, 3*n))
              + list(range(8*n, 8*n+1))
             )
    
    
    qc.append(controlledTwosComplement(n), 
              list(range(n, 2*n))
              + list(range(6*n, 7*n))
              + list(range(2*n, 3*n))
              + list(range(8*n+1, 8*n+2))
             )
    
    for i in range(n):
        # Setting Ancilla based on current bit of regA
        for j in range(n):
            qc.ccx(i, n+j, 2*n+i+j)
        qc.append(signedCummulativeAdder(n*2), range(2*n, 8*n))
        # Cleaning Ancilla
        for j in range(n):
            qc.ccx(i, n+j, 2*n+i+j)
            
    qc.append(controlledTwosComplement(n), 
              list(range(0, n))
              + list(range(6*n, 7*n))
              + list(range(2*n, 3*n))
              + list(range(8*n, 8*n+1))
             )
    
    
    qc.append(controlledTwosComplement(n), 
              list(range(n, 2*n))
              + list(range(6*n, 7*n))
              + list(range(2*n, 3*n))
              + list(range(8*n+1, 8*n+2))
             )
    # Checking if we need to invert the result
    qc.cx(8*n+1, 8*n)
    
    qc.append(controlledTwosComplement(2*n), 
              list(range(4*n, 6*n))
              + list(range(6*n, 8*n))
              + list(range(2*n, 4*n))
              + list(range(8*n, 8*n+1))
             )
    
    qc.cx(8*n+1, 8*n)
      
    qc.cx(n-1, 8*n)
    qc.cx(2*n-1, 8*n+1)
    if (inverse):
        U_Mult = qc.inverse().to_gate()
        U_Mult.name = "U_SignedMultInverse"
    else:
        U_Mult = qc.to_gate()
        U_Mult.name = "U_SignedMult"
    return U_Mult

"""
Oracle 1' Circuit

Computes the inner product of two row vectors of length k
Works with signed numbers
Brings the state |f>|d>|0> -> |f>|d>|(d.f)>

Qubits [0 : n*k] are for the row vector f
Qubits [n*k : 2*n*k] are for the row vector d
Qubits [2*n*k : 2*n*k + 2*n] are for carry bits
Qubits [2*n*k + 2*n : 2*n*k + 4*n] are for ancilla bits
Qubits [2*n*k + 4*n : 4*n*k + 4*n] are for the output registers
Qubits [4*n*k + 4*n : 4*n*k + 4*n + 2] are for control qubits

Total Qubits required = 4*n*k + 4*n + 2

Paramters: n

"""
def signedInnerProduct(n, k, inverse=False):
    totalQubits = 4*n*k + 4*n + 2
    qc = QuantumCircuit(totalQubits)
    for i in range(0, k):
        qc.append(signedMultiplier(n),
                              list(range(n*i, n*(i+1))) 
                            + list(range(n*(k+i), n*(k+i+1)))
                            + list(range(2*n*k + 2*n, 2*n*k + 4*n))
                            + list(range((2*n*k + 4*n + (2*n)*i), (2*n*k + 4*n + (2*n)*(i+1))))
                            + list(range(2*n*k, 2*n*k + 2*n))
                            + list(range(4*n*k + 4*n, 4*n*k + 4*n + 2))
                            )
        
    for i in range(1, k):
        qc.append(signedCummulativeAdder(n*2), 
                    list(range((2*n*k + 4*n + (2*n)*i), (2*n*k + 4*n + (2*n)*(i+1))))
                    + list(range(2*n*k + 4*n, 2*n*k + 4*n + 2*n))
                    + list(range(2*n*k, 2*n*k + 2*n))
                 )
    # Reverting all other output regs
    for i in range(1, k):
        qc.append(signedMultiplier(n, True),
                              list(range(n*i, n*(i+1))) 
                            + list(range(n*(k+i), n*(k+i+1)))
                            + list(range(2*n*k + 2*n, 2*n*k + 4*n))
                            + list(range((2*n*k + 4*n + (2*n)*i), (2*n*k + 4*n + (2*n)*(i+1))))
                            + list(range(2*n*k, 2*n*k + 2*n))
                            + list(range(4*n*k + 4*n, 4*n*k + 4*n + 2))
                            )
    if (inverse):
        U_Oracle1 = qc.inverse().to_gate()
        U_Oracle1.name = "U_Oracle1'Inverse"
    else:
        U_Oracle1 = qc.to_gate()
        U_Oracle1.name = "U_Oracle1'"
    return U_Oracle1

"""
Oracle 1 Circuit

Computes the square of the inner product of two row vectors of length k
Works with signed numbers
Brings the state |f>|d>|0> -> |f>|d>|(d.f)^2>

Qubits [0 : n*k] are for the row vector f
Qubits [n*k : 2*n*k] are for the row vector d
Qubits [2*n*k : 2*n*k + 4*n] are for carry bits
Qubits [2*n*k + 4*n : 2*n*k + 8*n] are for ancilla bits
Qubits [2*n*k + 8*n : 4*n*k + 8*n] are for the output registers
Qubits [4*n*k + 8*n : 4*n*k + 8*n + 2] are for control qubits
Qubits [4*n*k + 8*n + 2 : 4*n*k + 12*n + 2] are for the final output

Total Qubits required = 4*n*k + 12*n + 2

Paramters: n

"""
def oracle1(n, k, inverse=False):
    totalQubits = 4*n*k + 12*n + 2
    qc = QuantumCircuit(totalQubits)
    
    qc.append(signedInnerProduct(n, k, False), 
             list(range(0, 2*n*k + 2*n))
              + list(range(2*n*k + 4*n, 2*n*k + 6*n))
              + list(range(2*n*k + 8*n, 4*n*k + 8*n))
              + list(range(4*n*k + 8*n, 4*n*k + 8*n + 2))
             )
              
    for i in range(0, 2*n):
        qc.cx(2*n*k + 8*n + i, 2*n*k + 10*n + i)
            
    qc.append(signedMultiplier(2*n),
                              list(range(2*n*k + 8*n, 2*n*k + 10*n)) 
                            + list(range(2*n*k + 10*n, 2*n*k + 12*n))
                            + list(range(2*n*k + 4*n, 2*n*k + 8*n))
                            + list(range(4*n*k + 8*n + 2, 4*n*k + 12*n + 2))
                            + list(range(2*n*k, 2*n*k + 4*n))
                            + list(range(4*n*k + 8*n, 4*n*k + 8*n + 2))
                            )
    # Reverting ancilla bits to 0
    for i in range(0, 2*n):
        qc.cx(2*n*k + 8*n + i, 2*n*k + 10*n + i)
    qc.append(signedInnerProduct(n, k, True), 
             list(range(0, 2*n*k + 2*n))
              + list(range(2*n*k + 4*n, 2*n*k + 6*n))
              + list(range(2*n*k + 8*n, 4*n*k + 8*n))
              + list(range(4*n*k + 8*n, 4*n*k + 8*n + 2))
             )
              
    
    if (inverse):
        U_Oracle1 = qc.inverse().to_gate()
        U_Oracle1.name = "U_Oracle1Inverse"
    else:
        U_Oracle1 = qc.to_gate()
        U_Oracle1.name = "U_Oracle1"
    return U_Oracle1

"""
Greater than Equal Circuit

Compares two registers a and b and returns a >= b
Brings the state |a>|b>|0> -> |a>|b>|1> iff a >= b else to |a>|b>|0>

Qubits [0 : n] are for the register a
Qubits [n : 2*n] are for the register b
Qubits [2*n : 3*n] are for ancilla bits
Qubits [3*n : 3*n + 1] is for the output bit

Total Qubits required = 3*n + 1

Paramters: n

"""

# a >= b

def greaterThanEqual(n):
    totalQubits = 3*n + 1
    qc = QuantumCircuit(totalQubits)
    
    # Check most significant bit

    # Check that a[n-1] is 1 and b[n-1] is 0
    qc.x(2*n-1)
    qc.ccx(n-1, 2*n-1, 3*n)
    
    # Check if both a[n-1] and b[n-1] are 0 
    qc.x(n-1)
    qc.ccx(n-1, 2*n-1, 3*n-1)
    
    # Check if both a[n-1] and b[n-1] are 1
    qc.x(n-1)
    qc.x(2*n-1)
    qc.ccx(n-1, 2*n-1, 3*n-1)
    
    # Check the rest of the bits
    for i in range(n-2, -1, -1):
        # Check that a[i] is 1 and b[i] is 0 if the previous bits were equal
        qc.x(n+i)
        qc.mct([i, n+i, 2*n+i+1], 3*n)
    
        # Check if both a[i] and b[i] are 0 
        qc.x(i)
        qc.mct([i, n+i, 2*n+i+1], 2*n+i)
    
        # Check if both a[i] and b[i] are 1
        qc.x(i)
        qc.x(n+i)
        qc.mct([i, n+i, 2*n+i+1], 2*n+i)
        
    # Returning 1 if both numbers are equal
    qc.cx(2*n, 3*n)
    
    # Reversing circuit to put aux back to original state
    for i in range(0, n-1):
    
        # Check if both a[i] and b[i] are 0
        qc.x(n+i)
        qc.x(i)
        qc.mct([i, n+i, 2*n+i+1], 2*n+i)
    
        # Check if both a[i] and b[i] are 1
        qc.x(i)
        qc.x(n+i)
        qc.mct([i, n+i, 2*n+i+1], 2*n+i)
        
    qc.x(2*n-1)
    qc.x(n-1)
    qc.ccx(n-1, 2*n-1, 3*n-1)
    
    # Check if both a[n-1] and b[n-1] are 1
    qc.x(n-1)
    qc.x(2*n-1)
    qc.ccx(n-1, 2*n-1, 3*n-1)
    
    U_GreaterThan = qc.to_gate()
    U_GreaterThan.name = "U_GreaterThan"
    return U_GreaterThan

"""
Oracle 2

Compares 3 registers and returns alpha <= regA <= beta
Brings the state |alpha>|a>|beta>|0> -> 
                    |alpha>|a>|beta>|1> iff alpha <= regA <= beta else
                    |alpha>|a>|beta>|0>
Qubits [0 : n] are for the register a
Qubits [n : 2*n] are for the register alpha
Qubits [2*n : 3*n] are for the register beta
Qubits [3*n : 4*n+2] are for ancilla bits
Qubits [4*n+2 : 4*n+3] is for the output bit

Total Qubits required = 4*n+3

Paramters: n

"""

# a >= b

def oracle2(n):
    totalQubits = 4*n+3
    qc = QuantumCircuit(totalQubits)
    
    # Check if beta >= a, storing result in ancilla[4*n]
    qc.append(greaterThanEqual(n), list(range(2*n, 3*n))
                                + list(range(0, n))
                                + list(range(3*n, 4*n))
                                + list(range(4*n, 4*n+1))
            )
    # Check if alpha <= a, storing result in ancilla[4*n+1]
    qc.append(greaterThanEqual(n), list(range(0, n))
                                + list(range(n, 2*n))
                                + list(range(3*n, 4*n))
                                + list(range(4*n+1, 4*n+2))
            )
    # Check if both are true and return output
    qc.ccx(4*n, 4*n+1, 4*n+2)
    
    # Reset Ancilla Bits
    qc.append(greaterThanEqual(n), list(range(2*n, 3*n))
                                + list(range(0, n))
                                + list(range(3*n, 4*n))
                                + list(range(4*n, 4*n+1))
            )
    qc.append(greaterThanEqual(n), list(range(0, n))
                                + list(range(n, 2*n))
                                + list(range(3*n, 4*n))
                                + list(range(4*n+1, 4*n+2))
            )
    
    
    U_Oracle2 = qc.to_gate()
    U_Oracle2.name = "U_Oracle2"
    return U_Oracle2

def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

In [ ]:
# Putting Everything Together
iLength = 1
rows = 2**iLength
n = 2
k = 2

F = "0101" + "0111"
D = "0000" + "0101"

alpha = "00000100"
beta =  "00100000"

i = QuantumRegister(iLength, name="i")
j = QuantumRegister(iLength, name="j") 
 
fi = QuantumRegister(n*k, name="fi")
di = QuantumRegister(n*k, name="di")

f = QuantumRegister(n*k*rows, name="f")
d = QuantumRegister(n*k*rows, name="d")

ancilla = QuantumRegister(8*n + 2*n*k + 2, "ancilla")
oReg = QuantumRegister(4*n, "output")

# Qubits 2*iLength + 4*n*k + 2*n*k*rows + 12*n + 2 : 2*iLength + 4*n*k + 2*n*k*rows + 16*n + 2
regAlpha = QuantumRegister(4*n, name="alpha")
# Qubits 2*iLength + 4*n*k + 2*n*k*rows + 16*n + 2 : 2*iLength + 4*n*k + 2*n*k*rows + 20*n + 2
regBeta = QuantumRegister(4*n, name="beta")
kickbackReg = QuantumRegister(1, name="kickback")
output = ClassicalRegister(2+2*n)

qc = QuantumCircuit(i, j, fi, di, f, d, ancilla, oReg, regAlpha, regBeta, kickbackReg, output)

initilizeQubits(qc, F, f, n*k*rows)
initilizeQubits(qc, D, d, n*k*rows)
initilizeQubits(qc, alpha, regAlpha, 4*n)
initilizeQubits(qc, beta, regBeta, 4*n)

qc.h(i)
qc.h(j)
qc.x(kickbackReg)
qc.h(kickbackReg)


# print(list(regAlpha) + list(regBeta)) 
qc.append(load(n, iLength, k), 
                  list(i)
                + list(ancilla[0:iLength])
                + list(fi)
                + list(f)
         )

qc.append(load(n, iLength, k), 
                  list(j)
                + list(ancilla[0:iLength])
                + list(di)
                + list(d)
         )

qc.append(oracle1(n, k, False), 
                  list(fi)
                + list(di)
                + list(ancilla)
                + list(oReg)
         )

qc.append(oracle2(4*n), 
             list(oReg)
           + list(regAlpha)
           + list(regBeta)
           + list(ancilla[0:4*n+2])
           #+ list(ancilla[4*n+2:4*n+3])
           + list(kickbackReg)
         )

# qc.cx(ancilla[4*n+2], kickbackReg)

# qc.append(oracle2(4*n), 
#              list(oReg)
#            + list(regAlpha)
#            + list(regBeta)
#            + list(ancilla[0:4*n+2])
#            + list(ancilla[4*n+2:4*n+3])
#          )

qc.append(oracle1(n, k, True), 
                  list(fi)
                + list(di)
                + list(ancilla)
                + list(oReg)
         )

qc.append(load(n, iLength, k), 
                  list(j)
                + list(ancilla[0:iLength])
                + list(di)
                + list(d)
         )

qc.append(load(n, iLength, k), 
                  list(i)
                + list(ancilla[0:iLength])
                + list(fi)
                + list(f)
         )
qc.append(diffuser(2), list(i) + list(j))

qc.append(load(n, iLength, k), 
                  list(j)
                + list(ancilla[0:iLength])
                + list(di)
                + list(d)
         )

qc.append(load(n, iLength, k), 
                  list(i)
                + list(ancilla[0:iLength])
                + list(fi)
                + list(f)
         )

qc.append(signedInnerProduct(n, k, False),
             list(fi)
           + list(di)
           + list(ancilla[0:4*n])
           + list(oReg[0:2*n])
           + list(ancilla[4*n:4*n + 2 + 2*n*(k-1)])
         )
qc.x(i)
qc.x(j)
qc.measure(i, output[0])
qc.measure(j, output[1])
qc.measure(oReg[0:2*n], output[2:])
# qc.measure_all()
# qc.measure(range(2*iLength + 4*n*k + 2*n*k*rows + 8*n + 2*n*k + 2, 2*iLength + 4*n*k + 2*n*k*rows + 12*n + 2*n*k + 2), output)
qc.draw()


In [ ]:
# # Measure
# aer_sim = Aer.get_backend('aer_simulator')
# transpiled_grover_circuit = transpile(qc, aer_sim)
# qobj = assemble(transpiled_grover_circuit)
# results = aer_sim.run(qobj).result()
# counts = results.get_counts()
# print(counts)
from qiskit.providers.aer import AerSimulator
# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(qc, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

In [ ]:
"""
Load operation - Creates a superposition of all rows of a matrix

Takes the state |i>|0> --> |i>|fi> where fi is a row vectors of a matrix F

Puts the result into a superposition of all possible vector rows from F, where observing
|i> and |fi> will give you the row vectors fi .

Qubits [0 : iLength] hold the register for |i>
Qubits [iLength : 2*iLength] are ancilla bits for holding the NOT of |i>
Qubits [2*iLength : 2*iLength+(k*n)] hold the result vector |fi> and should initially be 0
Qubits [2*iLength + (k*n) : 2*iLength + n*k + n*k*2^iLength] hold the database for the matrix f
    
Parameters: iLength : Length of the register |i>
            n : Length of one element of the database
            k : Number of elements in a row of the database
            
Total qubits = 2*iLength + n*k + n*k*2^iLength
"""

def load(n, iLength, k):
    totalQubits = 2*iLength + n*k + n*k*2**iLength
    qc = QuantumCircuit(totalQubits)
    # ccx for every bit that requires a 1
    for i in range(0, iLength):
        qc.cx(i, i+iLength)
        qc.x(i+iLength)
        
    for i in range(0, 2**iLength):
        gates = []
        copy = i
        for j in range(iLength-1, -1, -1):
            if (copy - 2**j >= 0):
                copy -= 2**j
                gates.append(j)
            else:
                gates.append(j+iLength)
        for l in range(0, n*k):
            fPosition = 2*iLength + n*k + i*n*k + l
            gates.append(fPosition)
            
            qc.mct(gates, 2*iLength + l)
            
            gates.pop()      
        
    
    # Reverting the ancilla bits back to 0
    for i in range(0, iLength):
        qc.x(i+iLength)
        qc.cx(i, i+iLength)
        
    U_Load = qc.to_gate()
    U_Load.name = "U_Load"
#     print("Depth: ", qc.depth())
    return U_Load

k = 2
n = 2
iLength = 2

f0 = "1100" + "0110" + "1111" + "0110"

i = QuantumRegister(iLength, name="i")
iAnc = QuantumRegister(iLength, name="iAnc")

fi = QuantumRegister(n*k, name="fi")

f = QuantumRegister(n*k*2**iLength, name="f")

output = ClassicalRegister(iLength + n*k)

size = i.size + iAnc.size + fi.size + f.size
qc = QuantumCircuit(i, iAnc, fi, f, output)
qc.h(i)
initilizeQubits(qc, f0, f, n*k*2**iLength)
qc.append(load(n, iLength, k), range(size))
qc.x(i)

qc.measure(i, output[0:iLength])
qc.measure(fi, output[iLength:iLength+n*k])
qc.draw()

In [ ]:
from qiskit.providers.aer import AerSimulator
# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(qc, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

In [ ]:
"""
Cumulative Adder Circuit

Adds two n bit strings from reg A and B and store them in reg B
Reg B and Reg Cmust be of size n+1

Qubits [0 : n] are for regA
Qubits [n : 2*n] are for regB
Qubits [2*n : 3*n] are for the carry register regC

Total Qubits required = 3*n

Paramters: n

"""

def signedCummulativeAdder(n):
    totalQubits = 3*n
    qc = QuantumCircuit(totalQubits)
    
    # Adding A and B and Carry
    for i in range(n-1):
        qc.ccx(i, n+i, 2*n+1+i)
        qc.cx(i, n+i)
        qc.ccx(2*n+i, n+i, 2*n+1+i)
        
    qc.cx(n-1, 2*n-1)
    
    # Reversing regC to 0
    #Reversing the gate operation performed on b[n-1]
    qc.cx(3*n-1, 2*n-1)
    #Reversing the gate operations performed during the carry gate implementations
    #This is done to ensure the sum gates are fed with the correct input bit states
    for i in range(n-1):
        qc.ccx(3*n-2-i, 2*n-2-i, 3*n-1-i)
        qc.cx(n-2-i, 2*n-2-i)
        qc.ccx(n-2-i, 2*n-2-i, 3*n-1-i)
        #These two operations act as a sum gate; if a control bit is at                
        #the 1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(3*n-2-i, 2*n-2-i)
        qc.cx(n-2-i, 2*n-2-i)
        
    U_Adder = qc.to_gate()
    U_Adder.name = "U_SignedAdder"
#     print("Adder Depth:", qc.depth())
    return U_Adder

n = 3
a = "010"
b = "001"

# a = "111"
# b = "111"

# a = "001"
# b = "111"

# a = "011"
# b = "011"
totalQubits = 3*n

aReg = QuantumRegister(n, "a")
bReg = QuantumRegister(n, "b")
cReg = QuantumRegister(n, "carry")
output = ClassicalRegister(n)

qc = QuantumCircuit(aReg, bReg, cReg, output)

initilizeQubits(qc, a, aReg, n)
initilizeQubits(qc, b, bReg, n)

qc.append(signedCummulativeAdder(n), range(totalQubits))
qc.measure(bReg, output)
qc.draw()

In [ ]:
from qiskit.providers.aer import AerSimulator
# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(qc, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

In [ ]:
"""
Controlled 2's Complement Circuit

Given transform a bit string s into it's two's complement form -1, 
by inverting the bits and adding 1

Qubits [0 : n] are for the bit string s
Qubits [n : 2*n] are for carry bits for the addition
Qubits [2*n : 3*n] are for storing the 1 for the addition
Qubits [3*n : 3*n+1] are for storing the control bit

Total Qubits required = 3*n+1

Paramters: n

"""

def controlledTwosComplement(n):
    totalQubits = 3*n+1
    qc = QuantumCircuit(totalQubits)
    # Invert s iff control == 1
    for i in range(0, n):
        qc.cx(3*n, i)
    # Store the 1 iff control == 1
    qc.cx(3*n, 2*n)
    
    qc.append(signedCummulativeAdder(n), 
              list(range(2*n, 3*n))
              + list(range(0, n))
              + list(range(n, 2*n))
             )
    # Reverse the 1
    qc.cx(3*n, 2*n)
    U_TwosComplement = qc.to_gate()
    U_TwosComplement.name = "U_ControlledTwosComplement"
    return U_TwosComplement

n = 6
totalQubits = 3*n+1
a = "111111"
# a = "000001"

# a = "111001"

aReg = QuantumRegister(n, "a")
cReg = QuantumRegister(n, "carry")
ancReg = QuantumRegister(n, "ancilla")
control = QuantumRegister(1, "control")

output = ClassicalRegister(n)

qc = QuantumCircuit(aReg, cReg, ancReg, control, output)
qc.x(control)
initilizeQubits(qc, a, aReg, n)
qc.append(controlledTwosComplement(n), range(totalQubits))
qc.measure(aReg, output)
qc.draw()

In [ ]:
from qiskit.providers.aer import AerSimulator
# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(qc, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

In [ ]:
"""
Signed Shift Adder Multiplication Circuit

Multiplies two n bit registers regA and regB using a shift adder and
stores the result in a 2*n bit output register.

Qubits [0 : n] are for regA
Qubits [n : 2*n] are for regB
Qubits [2*n : 4*n] are for an ancilla register
Qubits [4*n : 6*n] are for the output register
Qubits [6*n : 8*n] are for the carry register regC
Qubits [8*n : 8*n+2] are for more ancilla bits

Total Qubits required = 8*n + 2

Paramters: n

"""

def signedMultiplier(n, inverse=False):
    totalQubits = 8*n+2
    qc = QuantumCircuit(totalQubits)
    
    # Checking if either are negative numbers
    
    qc.cx(n-1, 8*n)
    qc.cx(2*n-1, 8*n+1)
    
    qc.append(controlledTwosComplement(n), 
              list(range(0, n))
              + list(range(6*n, 7*n))
              + list(range(2*n, 3*n))
              + list(range(8*n, 8*n+1))
             )
    
    
    qc.append(controlledTwosComplement(n), 
              list(range(n, 2*n))
              + list(range(6*n, 7*n))
              + list(range(2*n, 3*n))
              + list(range(8*n+1, 8*n+2))
             )
    
    for i in range(n):
        # Setting Ancilla based on current bit of regA
        for j in range(n):
            qc.ccx(i, n+j, 2*n+i+j)
        qc.append(signedCummulativeAdder(n*2), range(2*n, 8*n))
        # Cleaning Ancilla
        for j in range(n):
            qc.ccx(i, n+j, 2*n+i+j)
            
    qc.append(controlledTwosComplement(n), 
              list(range(0, n))
              + list(range(6*n, 7*n))
              + list(range(2*n, 3*n))
              + list(range(8*n, 8*n+1))
             )
    
    
    qc.append(controlledTwosComplement(n), 
              list(range(n, 2*n))
              + list(range(6*n, 7*n))
              + list(range(2*n, 3*n))
              + list(range(8*n+1, 8*n+2))
             )
    # Checking if we need to invert the result
    qc.cx(8*n+1, 8*n)
    
    qc.append(controlledTwosComplement(2*n), 
              list(range(4*n, 6*n))
              + list(range(6*n, 8*n))
              + list(range(2*n, 4*n))
              + list(range(8*n, 8*n+1))
             )
    
    qc.cx(8*n+1, 8*n)
      
    qc.cx(n-1, 8*n)
    qc.cx(2*n-1, 8*n+1)
    if (inverse):
        U_Mult = qc.inverse().to_gate()
        U_Mult.name = "U_SignedMultInverse"
    else:
        U_Mult = qc.to_gate()
        U_Mult.name = "U_SignedMult"
    return U_Mult

n = 4
a = "0011"
b = "0011"

# a = "0010"
# b = "1110"

# a = "1110"
# b = "1110"

# a = "1000"
# b = "0000"
totalQubits = 8*n+2

aReg = QuantumRegister(n, "a")
bReg = QuantumRegister(n, "b")
ancReg = QuantumRegister(2*n, "ancilla")
oReg = QuantumRegister(2*n, "o")
cReg = QuantumRegister(2*n, "carry")
control = QuantumRegister(2, "control")
output = ClassicalRegister(2*n)

qc = QuantumCircuit(aReg, bReg, ancReg, oReg, cReg, control, output)

initilizeQubits(qc, a, aReg, n)
initilizeQubits(qc, b, bReg, n)

qc.append(signedMultiplier(n), range(totalQubits))
qc.measure(oReg, output)
qc.draw()

In [ ]:
from qiskit.providers.aer import AerSimulator
# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(qc, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

In [ ]:
"""
Oracle 1' Circuit

Computes the inner product of two row vectors of length k
Works with signed numbers
Brings the state |f>|d>|0> -> |f>|d>|(d.f)>

Qubits [0 : n*k] are for the row vector f
Qubits [n*k : 2*n*k] are for the row vector d
Qubits [2*n*k : 2*n*k + 2*n] are for carry bits
Qubits [2*n*k + 2*n : 2*n*k + 4*n] are for ancilla bits
Qubits [2*n*k + 4*n : 4*n*k + 4*n] are for the output registers
Qubits [4*n*k + 4*n : 4*n*k + 4*n + 2] are for control qubits

Total Qubits required = 4*n*k + 4*n + 2

Paramters: n

"""
def signedInnerProduct(n, k, inverse=False):
    totalQubits = 4*n*k + 4*n + 2
    qc = QuantumCircuit(totalQubits)
    for i in range(0, k):
        qc.append(signedMultiplier(n),
                              list(range(n*i, n*(i+1))) 
                            + list(range(n*(k+i), n*(k+i+1)))
                            + list(range(2*n*k + 2*n, 2*n*k + 4*n))
                            + list(range((2*n*k + 4*n + (2*n)*i), (2*n*k + 4*n + (2*n)*(i+1))))
                            + list(range(2*n*k, 2*n*k + 2*n))
                            + list(range(4*n*k + 4*n, 4*n*k + 4*n + 2))
                            )
        
    for i in range(1, k):
        qc.append(signedCummulativeAdder(n*2), 
                    list(range((2*n*k + 4*n + (2*n)*i), (2*n*k + 4*n + (2*n)*(i+1))))
                    + list(range(2*n*k + 4*n, 2*n*k + 4*n + 2*n))
                    + list(range(2*n*k, 2*n*k + 2*n))
                 )
    # Reverting all other output regs
    for i in range(1, k):
        qc.append(signedMultiplier(n, True),
                              list(range(n*i, n*(i+1))) 
                            + list(range(n*(k+i), n*(k+i+1)))
                            + list(range(2*n*k + 2*n, 2*n*k + 4*n))
                            + list(range((2*n*k + 4*n + (2*n)*i), (2*n*k + 4*n + (2*n)*(i+1))))
                            + list(range(2*n*k, 2*n*k + 2*n))
                            + list(range(4*n*k + 4*n, 4*n*k + 4*n + 2))
                            )
    if (inverse):
        U_Oracle1 = qc.inverse().to_gate()
        U_Oracle1.name = "U_Oracle1'Inverse"
    else:
        U_Oracle1 = qc.to_gate()
        U_Oracle1.name = "U_Oracle1'"
    return U_Oracle1

n = 3
k = 3
totalQubits = 4*n*k + 4*n + 2
fAll = "111" + "100" + "011"
dAll = "010" + "110" + "001"
# Should equal 9

# fAll = "111" + "011" + "100"
# dAll = "110" + "110" + "011"
# Should equal -16

f = QuantumRegister(n*k, "f")
d = QuantumRegister(n*k, "d")
cReg = QuantumRegister(2*n, "carry")
ancReg = QuantumRegister(2*n, "ancilla")
oReg = QuantumRegister((2*n) * k, "o")
control = QuantumRegister(2, "control")

output = ClassicalRegister(2*n)

qc = QuantumCircuit(f, d, cReg, ancReg, oReg, control, output)

initilizeQubits(qc, fAll, f, n*k)
initilizeQubits(qc, dAll, d, n*k)
qc.append(signedInnerProduct(n, k, False), range(totalQubits))
# qc.append(signedInnerProduct(n, k, True), range(totalQubits))
# print(qc.data)
qc.measure(oReg[0:2*n], output)
qc.draw()

In [ ]:
from qiskit.providers.aer import AerSimulator
# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(qc, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

In [ ]:
"""
Oracle 1 Circuit

Computes the square of the inner product of two row vectors of length k
Works with signed numbers
Brings the state |f>|d>|0> -> |f>|d>|(d.f)^2>

Qubits [0 : n*k] are for the row vector f
Qubits [n*k : 2*n*k] are for the row vector d
Qubits [2*n*k : 2*n*k + 4*n] are for carry bits
Qubits [2*n*k + 4*n : 2*n*k + 8*n] are for ancilla bits
Qubits [2*n*k + 8*n : 4*n*k + 8*n] are for the output registers
Qubits [4*n*k + 8*n : 4*n*k + 8*n + 2] are for control qubits
Qubits [4*n*k + 8*n + 2 : 4*n*k + 12*n + 2] are for the final output

Total Qubits required = 4*n*k + 12*n + 2

Paramters: n

"""
def oracle1(n, k, inverse=False):
    totalQubits = 4*n*k + 12*n + 2
    qc = QuantumCircuit(totalQubits)
    
    qc.append(signedInnerProduct(n, k, False), 
             list(range(0, 2*n*k + 2*n))
              + list(range(2*n*k + 4*n, 2*n*k + 6*n))
              + list(range(2*n*k + 8*n, 4*n*k + 8*n))
              + list(range(4*n*k + 8*n, 4*n*k + 8*n + 2))
             )
              
    for i in range(0, 2*n):
        qc.cx(2*n*k + 8*n + i, 2*n*k + 10*n + i)
            
    qc.append(signedMultiplier(2*n),
                              list(range(2*n*k + 8*n, 2*n*k + 10*n)) 
                            + list(range(2*n*k + 10*n, 2*n*k + 12*n))
                            + list(range(2*n*k + 4*n, 2*n*k + 8*n))
                            + list(range(4*n*k + 8*n + 2, 4*n*k + 12*n + 2))
                            + list(range(2*n*k, 2*n*k + 4*n))
                            + list(range(4*n*k + 8*n, 4*n*k + 8*n + 2))
                            )
    # Reverting ancilla bits to 0
    for i in range(0, 2*n):
        qc.cx(2*n*k + 8*n + i, 2*n*k + 10*n + i)
    qc.append(signedInnerProduct(n, k, True), 
             list(range(0, 2*n*k + 2*n))
              + list(range(2*n*k + 4*n, 2*n*k + 6*n))
              + list(range(2*n*k + 8*n, 4*n*k + 8*n))
              + list(range(4*n*k + 8*n, 4*n*k + 8*n + 2))
             )
              
    
    if (inverse):
        U_Oracle1 = qc.inverse().to_gate()
        U_Oracle1.name = "U_Oracle1Inverse"
    else:
        U_Oracle1 = qc.to_gate()
        U_Oracle1.name = "U_Oracle1"
    return U_Oracle1

n = 2
k = 2
totalQubits = 4*n*k + 12*n + 2
fAll = "10" + "01"
dAll = "01" + "01"
# Should be 1**2 = 1

# fAll = "10" + "10"
# dAll = "10" + "10"
# Should be 8*8 = 64

f = QuantumRegister(n*k, "f")
d = QuantumRegister(n*k, "d")
cReg = QuantumRegister(4*n, "carry")
ancReg = QuantumRegister(4*n, "ancilla")
oReg = QuantumRegister((2*n) * k, "o")
control = QuantumRegister(2, "control")
finalO = QuantumRegister((4*n), "final")

output = ClassicalRegister(4*n)

qc = QuantumCircuit(f, d, cReg, ancReg, oReg, control, finalO, output)

initilizeQubits(qc, fAll, f, n*k)
initilizeQubits(qc, dAll, d, n*k)

qc.append(oracle1(n, k, False), range(totalQubits))
qc.measure(finalO, output)
qc.draw()

In [ ]:
from qiskit.providers.aer import AerSimulator
# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(qc, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

In [ ]:
"""
Greater than Equal Circuit - DOESN'T WORK WITH SIGNED NUMBERS

Compares two registers a and b and returns a >= b
Brings the state |a>|b>|0> -> |a>|b>|1> iff a >= b else to |a>|b>|0>

Qubits [0 : n] are for the register a
Qubits [n : 2*n] are for the register b
Qubits [2*n : 3*n] are for ancilla bits
Qubits [3*n : 3*n + 1] is for the output bit

Total Qubits required = 3*n + 1

Paramters: n

"""

# a >= b

def greaterThanEqual(n):
    totalQubits = 3*n + 1
    qc = QuantumCircuit(totalQubits)
    
    # Check most significant bit

    # Check that a[n-1] is 1 and b[n-1] is 0
    qc.x(2*n-1)
    qc.ccx(n-1, 2*n-1, 3*n)
    
    # Check if both a[n-1] and b[n-1] are 0 
    qc.x(n-1)
    qc.ccx(n-1, 2*n-1, 3*n-1)
    
    # Check if both a[n-1] and b[n-1] are 1
    qc.x(n-1)
    qc.x(2*n-1)
    qc.ccx(n-1, 2*n-1, 3*n-1)
    
    # Check the rest of the bits
    for i in range(n-2, -1, -1):
        # Check that a[i] is 1 and b[i] is 0 if the previous bits were equal
        qc.x(n+i)
        qc.mct([i, n+i, 2*n+i+1], 3*n)
    
        # Check if both a[i] and b[i] are 0 
        qc.x(i)
        qc.mct([i, n+i, 2*n+i+1], 2*n+i)
    
        # Check if both a[i] and b[i] are 1
        qc.x(i)
        qc.x(n+i)
        qc.mct([i, n+i, 2*n+i+1], 2*n+i)
        
    # Returning 1 if both numbers are equal
    qc.cx(2*n, 3*n)
    
    # Reversing circuit to put aux back to original state
    for i in range(0, n-1):
    
        # Check if both a[i] and b[i] are 0
        qc.x(n+i)
        qc.x(i)
        qc.mct([i, n+i, 2*n+i+1], 2*n+i)
    
        # Check if both a[i] and b[i] are 1
        qc.x(i)
        qc.x(n+i)
        qc.mct([i, n+i, 2*n+i+1], 2*n+i)
        
    qc.x(2*n-1)
    qc.x(n-1)
    qc.ccx(n-1, 2*n-1, 3*n-1)
    
    # Check if both a[n-1] and b[n-1] are 1
    qc.x(n-1)
    qc.x(2*n-1)
    qc.ccx(n-1, 2*n-1, 3*n-1)
    
    U_GreaterThan = qc.to_gate()
    U_GreaterThan.name = "U_GreaterThan"
    return U_GreaterThan

a = "0001"
b = "0100"

# a = "0100"
# b = "0010"

# a = "0110"
# b = "0110"

n = 4
totalQubits = 3*n + 1

regA = QuantumRegister(n, name="a")
regB = QuantumRegister(n, name="b")
regAux = QuantumRegister(n, name="aux")
regO = QuantumRegister(1, name="o")

output = ClassicalRegister(1)

qc = QuantumCircuit(regA, regB, regAux, output, regO)

initilizeQubits(qc, b, regB, n)
initilizeQubits(qc, a, regA, n)

qc.append(greaterThanEqual(n), range(totalQubits))
qc.barrier()
qc.measure(regO, output)
qc.draw()

In [ ]:
from qiskit.providers.aer import AerSimulator
# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(qc, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts

In [ ]:
"""
Oracle 2

Compares 3 registers and returns alpha <= regA <= beta
Brings the state |alpha>|a>|beta>|0> -> 
                    |alpha>|a>|beta>|1> iff alpha <= regA <= beta else
                    |alpha>|a>|beta>|0>
Qubits [0 : n] are for the register a
Qubits [n : 2*n] are for the register alpha
Qubits [2*n : 3*n] are for the register beta
Qubits [3*n : 4*n+2] are for ancilla bits
Qubits [4*n+2 : 4*n+3] is for the output bit

Total Qubits required = 4*n+3

Paramters: n

"""



def oracle2(n):
    totalQubits = 4*n+3
    qc = QuantumCircuit(totalQubits)
    
    # Check if beta >= a, storing result in ancilla[4*n]
    qc.append(greaterThanEqual(n), list(range(2*n, 3*n))
                                + list(range(0, n))
                                + list(range(3*n, 4*n))
                                + list(range(4*n, 4*n+1))
            )
    # Check if alpha <= a, storing result in ancilla[4*n+1]
    qc.append(greaterThanEqual(n), list(range(0, n))
                                + list(range(n, 2*n))
                                + list(range(3*n, 4*n))
                                + list(range(4*n+1, 4*n+2))
            )
    # Check if both are true and return output
    qc.ccx(4*n, 4*n+1, 4*n+2)
    
    # Reset Ancilla Bits
    qc.append(greaterThanEqual(n), list(range(2*n, 3*n))
                                + list(range(0, n))
                                + list(range(3*n, 4*n))
                                + list(range(4*n, 4*n+1))
            )
    qc.append(greaterThanEqual(n), list(range(0, n))
                                + list(range(n, 2*n))
                                + list(range(3*n, 4*n))
                                + list(range(4*n+1, 4*n+2))
            )
    
    
    U_Oracle2 = qc.to_gate()
    U_Oracle2.name = "U_Oracle2"
    return U_Oracle2

alpha = "0001"
a = "0100"
beta = "0111"

# alpha = "0001"
# a = "0100"
# beta = "0011"


# alpha = "0111"
# a = "0111"
# beta = "0111"
n = 4
totalQubits = 4*n + 3

regA = QuantumRegister(n, name="a")
regAlpha = QuantumRegister(n, name="alpha")
regBeta = QuantumRegister(n, name="beta")
regAnc = QuantumRegister(n+2, name="anc")
regO = QuantumRegister(1, name="o")

output = ClassicalRegister(1)

qc = QuantumCircuit(regA, regAlpha, regBeta, regAnc, regO, output)


initilizeQubits(qc, a, regA, n)
initilizeQubits(qc, alpha, regAlpha, n)
initilizeQubits(qc, beta, regBeta, n)

qc.append(oracle2(n), range(totalQubits))
qc.barrier()
qc.measure(regO, output)
qc.draw()

In [ ]:
from qiskit.providers.aer import AerSimulator
# Select the AerSimulator from the Aer provider
simulator = AerSimulator(method='matrix_product_state')

# Run and get counts, using the matrix_product_state method
tcirc = transpile(qc, simulator)
result = simulator.run(tcirc).result()
counts = result.get_counts(0)
counts